In [105]:
import numpy as np
import os 
import glob
from astropy.io import fits
import matplotlib.pyplot as plt
import itertools
import sys

%matplotlib notebook

# ----------------- Import the other files of functions
module_path = os.path.join('/Users/jennifer/Desktop/ASAP')
if module_path not in sys.path:
    sys.path.append(module_path)
    
import asap_lib.spectra as sa

In [106]:
path = '../GHOST_ObsData/Sag2_IFU1/output/'
spath = path+'Sag2_IFU1'
IFU = 0

In [111]:
for camera in ['blue', 'red']:

    condition = '*{}*Output.fits'.format(camera)
    obs = glob.glob(path+condition)

    # --------------------------------
    waves = []
    fluxes = []
    errs = []

    for item in obs:
        spectra = fits.open(item)

        wave = spectra[7].data
        flux = spectra[5].data[:,IFU]
        err = spectra[6].data[:,IFU]

        waves.append(wave)
        fluxes.append(flux)
        errs.append(err)

    # --------------------------------
    pairs =  list(itertools.combinations(range(len(waves)), 2))

    for pair in pairs:
        value = np.array_equal(waves[pair[0]], waves[pair[1]])
        if value == False:
            print('Wavelength arrays not equivalent. Do not proceed')
            break

    if len(waves) > 0:
        # -------------------------------- Median Combine flux and combine error with Poisson statistics 
        finalWave = waves[0]
        finalFlux = np.median(fluxes,axis=0)
        finalErr = np.sqrt( sum(i*i for i in errs) ) / len(errs) 


        # -------------------------------- Save the output as .xy and .bin files 
        sa.write2xy(finalWave, finalFlux, finalErr, spath+'_{}'.format(camera))
        sa.write2bin(finalWave, finalFlux, finalErr, spath+'_{}'.format(camera))

        print('Saved combination of {} {} camera images'.format(len(waves), camera))
        # --------------------------------

    else:
        print('No {} camera files found'.format(camera))

Saved combination of 3 blue camera images
No red camera files found
